In [29]:
#import 
from bs4 import BeautifulSoup
import pandas as pd
import os
import chardet
import re

In [1]:
def get_all_files(path):
    files = []
    for file in os.listdir(path):
        if file.endswith('.xml'):
            files.append(file)
    return files

In [4]:
def extract_data(file):
    soup =detect_encoding(file)
    auteur = soup.find('sourceDesc').find('author').get_text()
    titre = soup.find('sourceDesc').find('title').get_text
    publisher = soup.find('distributor').get_text()
    date = soup.find('sourceDesc').find('date').get_text()
    categorie = soup.find('textClass').find('keywords', attr={'scheme':'genre'}).get_text()
    abstract = soup.find('div', attr={'type':'abstract'}).get_text()
    body = soup.find('body')
    text = ' '.join([p.get_text() for p in body.find_all('p')])
    return auteur, titre, publisher, date, categorie, abstract, text

In [5]:
def extract_data2(file):
    soup = detect_encoding(file)
    try : 
        auteur = soup.find('sourceDesc').find('author').get_text().strip()
    except : 
        auteur = ''
    try : 
        titre = soup.find('sourceDesc').find('title').get_text().strip()
    except : 
        titre = ''
    try : 
        publisher = soup.find('publicationStmt').find('publisher').get_text().replace('\n', ' ').strip()
    except : 
        publisher = ''
    try : 
        date = soup.find('sourceDesc').find('date').get_text().strip()
    except : 
        date = ''
    try : 
        categorie = soup.find('textClass').find('keywords', scheme='genre').get_text().strip()
    except : 
        categorie = ''
    try : 
        abstract = soup.find('div', type='abstract').get_text().replace('\n', ' ').strip()
    except : 
        abstract = ''
    body = soup.find('body')
    try : 
        text = ' '.join([p.get_text().strip() for p in body.find_all('p')])
        text = clean(text)
    except : 
        text = ''
    return auteur, titre, publisher, date, categorie, abstract, text


In [6]:
def detect_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding'] 
    # Open the file with the detected encoding
    with open(file, 'r', encoding=encoding) as f:
        soup = BeautifulSoup(f, "lxml-xml")
    return soup

In [36]:
def clean(text):
    text = text.replace('\xa0',' ')
    text = text.replace('\n                ','')
    text = text.replace('        ', ' ')
    text = text.replace('     ', ' ')
    text = text.replace('    ', ' ')
    text = re.sub('\[\d*\]', '', text)
    text = text.replace('\xad', '').replace('�', '')
    text = text.replace('\"', '')
    text = text.replace('[?]', '').replace('\n','').replace('\t','')
    return text

In [3]:
#extract all files from the folder
path = r"F:\Corpus\Corpus Scientext Fr-1.0 Licence CC"
all_files = get_all_files(path)

In [38]:
auteurs = []
titres = []
publishers = []
dates = []
categories = []
abstracts = []
texts = []
#extract all data
for x in all_files:
    auteur, titre, publisher, date, categorie, abstract, text = extract_data2("F:\\Corpus\\Corpus Scientext Fr-1.0 Licence CC\\"+x)
    auteurs.append(auteur)
    titres.append(titre)
    publishers.append(publisher)
    dates.append(date)
    categories.append(categorie)
    abstracts.append(abstract)
    text = clean(text)
    texts.append(text)

In [39]:
#store it in a df
df = pd.DataFrame(list(zip(auteurs, titres, publishers, dates, categories, abstracts, texts)),
              columns=['auteur', 'titre', 'publisher', 'date', 'categorie', 'abstract', 'text'])

In [40]:
#create 2 df 
df3 = df[['text']]
df2 = df[['titre','auteur','text']]

In [41]:
#store them in folder 
df3.to_csv(r"F:\Corpus\finaux\Karo\scientext.tsv", sep='\t', encoding='utf8', index=False)
df2.to_csv(r"F:\Corpus\finaux\Karo\scientext_3col.tsv", sep='\t', encoding='utf8', index=False)